In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import json
import xgcm

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

In [ ]:
def get_stuff_at_y(ht, y, mld, drf):
    HT = ht.isel(YG=y)
    MLD = mld.isel(YG=y).values
    HT_in_ML = HT.where(HT.Z >= -MLD, other=0)
    drF_in_ML = drf.where(HT.Z.values >= -MLD, other=0)
    HT_total_ML = (HT_in_ML * drF_in_ML.data).sum("Z")
    HT_out_ML = HT.where(HT.Z < -MLD, other=0)
    return MLD, HT_total_ML, HT_out_ML

In [ ]:
JJA = True

In [ ]:
if JJA:
    Ms = xr.open_mfdataset(eddypath + 'MHTs.JJA.0201-0300.nc')
    MsML = xr.open_mfdataset(eddypath + 'MHTsML.JJA.0201-0300.nc')
    MsNoML = xr.open_mfdataset(eddypath + 'MHTsNoML.JJA.0201-0300.nc')
    Ts = xr.open_mfdataset(eddypath + 'THTs.JJA.0201-0300.nc').drop("z")
    TsML = xr.open_mfdataset(eddypath + 'THTsML.JJA.0201-0300.nc').drop("z")
    TsNoML = xr.open_mfdataset(eddypath + 'THTsNoML.JJA.0201-0300.nc').drop("z")
    VHT = xr.open_mfdataset(path + "post/VHT_across_base_of_winter_MLD.JJA.0201-0300.nc").rename({"lat": "YG"})
    OIQ = xr.open_mfdataset(path + "post/OIQ.JJA.0201-0300.nc")
else:
    Ms = xr.open_mfdataset(eddypath + 'MHTs.0201-0300.nc')
    MsML = xr.open_mfdataset(eddypath + 'MHTsML.0201-0300.nc')
    MsNoML = xr.open_mfdataset(eddypath + 'MHTsNoML.0201-0300.nc')
    Ts = xr.open_mfdataset(eddypath + 'THTs.0201-0300.nc').drop("z")
    TsML = xr.open_mfdataset(eddypath + 'THTsML.0201-0300.nc').drop("z")
    TsNoML = xr.open_mfdataset(eddypath + 'THTsNoML.0201-0300.nc').drop("z")
    VHT = xr.open_mfdataset(path + "post/VHT_across_base_of_winter_MLD.0201-0300.nc").rename({"lat": "YG"})
    OIQ = xr.open_mfdataset(path + "post/OIQ.JJA.0201-0300.nc")

In [ ]:
data = xr.open_zarr(path + "zarr_Diags/output.5d.zarr").sel(time=slice("0201-01-01", "0300-12-30"))

In [ ]:
metrics = {
        ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
        ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
        ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
        ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
        }
grid = xgcm.Grid(data, periodic=["X"], metrics=metrics)

In [ ]:
meanJJAMLDxy = data.MXLDEPTH.groupby("time.season").mean("time").sel(season="JJA").mean("XC")
meanJJAMLD = grid.interp(meanJJAMLDxy, "Y", boundary="extend").compute()

In [ ]:
thickJJAxy = data.SIheff.groupby("time.season").mean("time").sel(season="JJA").mean("XC")
thickJJA = grid.interp(thickJJAxy, "Y", boundary="extend").compute()
areaJJAxy = data.SIarea.groupby("time.season").mean("time").sel(season="JJA").mean("XC")
areaJJA = grid.interp(areaJJAxy, "Y", boundary="extend").compute()

In [ ]:
ice_edge15 = 320 - areaJJA.where(areaJJA <= 0.15, other=1)[::-1].argmax("YG").values
ice_edge80 = 320 - areaJJA.where(areaJJA <= 0.80, other=1)[::-1].argmax("YG").values

In [ ]:
ML_at_190, MHT_ML_at_190, MHT_outside_ML_190 = get_stuff_at_y(Ms.MHT, 190, meanJJAMLD, data.drF)
ML_at_190, THTdiv_ML_at_190, THTdiv_outside_ML_190 = get_stuff_at_y(Ts.THTdiv, 190, meanJJAMLD, data.drF)
ML_at_190, THTdiveddy_ML_at_190, THTdiveddy_outside_ML_190 = get_stuff_at_y(Ts.THTdiveddy, 190, meanJJAMLD, data.drF)
ML_at_190, total_ML_at_190, total_outside_ML_190 = get_stuff_at_y(Ms.MHT + Ts.THTdiv, 190, meanJJAMLD, data.drF)

In [ ]:
ML_at_ice15, MHT_ML_at_ice15, MHT_outside_ML_ice15 = get_stuff_at_y(Ms.MHT, ice_edge15, meanJJAMLD, data.drF)
ML_at_ice15, THTdiv_ML_at_ice15, THTdiv_outside_ML_ice15 = get_stuff_at_y(Ts.THTdiv, ice_edge15, meanJJAMLD, data.drF)
ML_at_ice15, THTdiveddy_ML_at_ice15, THTdiveddy_outside_ML_ice15 = get_stuff_at_y(Ts.THTdiveddy, ice_edge15, meanJJAMLD, data.drF)
ML_at_ice15, total_ML_at_ice15, total_outside_ML_ice15 = get_stuff_at_y(Ms.MHT + Ts.THTdiv, ice_edge15, meanJJAMLD, data.drF)

In [ ]:
ML_at_ice80, MHT_ML_at_ice80, MHT_outside_ML_ice80 = get_stuff_at_y(Ms.MHT, ice_edge80, meanJJAMLD, data.drF)
ML_at_ice80, THTdiv_ML_at_ice80, THTdiv_outside_ML_ice80 = get_stuff_at_y(Ts.THTdiv, ice_edge80, meanJJAMLD, data.drF)
ML_at_ice80, THTdiveddy_ML_at_ice80, THTdiveddy_outside_ML_ice80 = get_stuff_at_y(Ts.THTdiveddy, ice_edge80, meanJJAMLD, data.drF)
ML_at_ice80, total_ML_at_ice80, total_outside_ML_ice80 = get_stuff_at_y(Ms.MHT + Ts.THTdiv, ice_edge80, meanJJAMLD, data.drF)

In [ ]:
ML_at_80, MHT_ML_at_80, MHT_outside_ML_80 = get_stuff_at_y(Ms.MHT, 80, meanJJAMLD, data.drF)
ML_at_80, THTdiv_ML_at_80, THTdiv_outside_ML_80 = get_stuff_at_y(Ts.THTdiv, 80, meanJJAMLD, data.drF)
ML_at_80, THTdiveddy_ML_at_80, THTdiveddy_outside_ML_80 = get_stuff_at_y(Ts.THTdiveddy, 80, meanJJAMLD, data.drF)
ML_at_80, total_ML_at_80, total_outside_ML_80 = get_stuff_at_y(Ms.MHT + Ts.THTdiv, 80, meanJJAMLD, data.drF)

In [ ]:
def print_results(total, mean, transient, eddy, text):
    output = {}
    output["total_mean"] =  np.around(total.mean("time").values / 1e10) / 100
    output["total_std"] = np.around(total.std("time").values / 1e10) / 100
    output["mean_mean"] = np.around(mean.mean("time").values / 1e10) / 100
    output["mean_std"] = np.around(mean.std("time").values / 1e10) / 100
    output["transient_mean"] = np.around(transient.mean("time").values / 1e10) / 100
    output["transient_std"] = np.around(transient.std("time").values / 1e10) / 100
    output["CME_mean"] = np.around(eddy.mean("time").values / 1e10) / 100
    output["CME_std"] = np.around(eddy.std("time").values / 1e10) / 100
    output["CME_percent"] = np.around(eddy.mean("time").values / transient.mean("time").values * 1000) / 10
    print(text, "is \n", 
          str(output["total_mean"]),
          "+-", str(output["total_std"]), "TW\n",
          " contribution by the mean is\n  ", str(output["mean_mean"]),
          "+-", str(output["mean_std"]), "TW\n",
          " contribution by the transient is\n  ", str(output["transient_mean"]),
          "+-", str(output["transient_std"]), "TW\n",
          "  to that, CME contribute\n   ", str(output["CME_mean"]),
          "+-", str(output["CME_std"]), "TW\n",
          "    or", str(output["CME_percent"]), "%")
    return output

In [ ]:
output = {}
output["horizontal HT within ML"] = {}
output["horizontal HT within ML"]["at y=190"] = print_results(total_ML_at_190, MHT_ML_at_190, 
                                                             THTdiv_ML_at_190, THTdiveddy_ML_at_190, 
                                                             "heat transport within the ML at y=190")

output["horizontal HT within ML"]["at 15% ice"] = print_results(total_ML_at_ice15, MHT_ML_at_ice15, 
                                                                THTdiv_ML_at_ice15, THTdiveddy_ML_at_ice15,
                                                                "heat transport within the ML at the 15% ice edge")

output["horizontal HT within ML"]["at 80% ice"] = print_results(total_ML_at_ice80, MHT_ML_at_ice80, 
                                                                THTdiv_ML_at_ice80, THTdiveddy_ML_at_ice80, 
                                                                "heat transport within the ML at the 80% ice edge")

output["horizontal HT within ML"]["at y=80"] = print_results(total_ML_at_80, MHT_ML_at_80, 
                                                             THTdiv_ML_at_80, THTdiveddy_ML_at_80, 
                                                             "heat transport within the ML at y=80")

output["horizontal HT below ML"] = {}
output["horizontal HT below ML"]["at y=190"] = print_results((total_outside_ML_190 * data.drF.data).sum("Z"), 
                                                             (MHT_outside_ML_190 * data.drF.data).sum("Z"),
                                                             (THTdiv_outside_ML_190 * data.drF.data).sum("Z"), 
                                                             (THTdiveddy_outside_ML_190 * data.drF.data).sum("Z"), 
                                                             "heat transport below the ML at y=190")

output["horizontal HT below ML"]["at 15% ice"] = print_results((total_outside_ML_ice15 * data.drF.data).sum("Z"), 
                                                               (MHT_outside_ML_ice15 * data.drF.data).sum("Z"), 
                                                               (THTdiv_outside_ML_ice15 * data.drF.data).sum("Z"), 
                                                               (THTdiveddy_outside_ML_ice15 * data.drF.data).sum("Z"), 
                                                               "heat transport below the ML at 15% ice")

output["horizontal HT below ML"]["at 80% ice"] = print_results((total_outside_ML_ice80 * data.drF.data).sum("Z"), 
                                                               (MHT_outside_ML_ice80 * data.drF.data).sum("Z"), 
                                                               (THTdiv_outside_ML_ice80 * data.drF.data).sum("Z"), 
                                                               (THTdiveddy_outside_ML_ice80 * data.drF.data).sum("Z"), 
                                                               "heat transport below the ML at 80% ice")

output["horizontal HT below ML"]["at y=80"] = print_results((total_outside_ML_80 * data.drF.data).sum("Z"), 
                                                            (MHT_outside_ML_80 * data.drF.data).sum("Z"), 
                                                            (THTdiv_outside_ML_80 * data.drF.data).sum("Z"), 
                                                            (THTdiveddy_outside_ML_80 * data.drF.data).sum("Z"), 
                                                            "heat transport below the ML at y=80")

In [ ]:
def calc_vertical(data, ds, diff=False):
    if diff:
        fifteenice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge15))).sum("YG")
        eightyice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge80))).sum("YG")
        eighty = (data.where(ds.YG < ds.YG.isel(YG=80))).sum("YG")
    else:
        fifteenice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge15)) * ds.dyU.isel(XG=0)).sum("YG")
        eightyice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge80)) * ds.dyU.isel(XG=0)).sum("YG")
        eighty = (data.where(ds.YG < ds.YG.isel(YG=80)) * ds.dyU.isel(XG=0)).sum("YG")
    fifteenice2eightyice = fifteenice - eightyice
    eightyice2eighty = eightyice - eighty
    eighty2zero = eighty 
    return fifteenice2eightyice, eightyice2eighty, eighty2zero

In [ ]:
fice2eice_total, eice2e_total, e2zero_total = calc_vertical(VHT.VHTbar_ice15 + VHT.VHTprime_ice15, data)
fice2eice_bar, eice2e_bar, e2zero_bar = calc_vertical(VHT.VHTbar_ice15, data)
fice2eice_prime, eice2e_prime, e2zero_prime = calc_vertical(VHT.VHTprime_ice15, data)
fice2eice_eddy, eice2e_eddy, e2zero_eddy = calc_vertical(VHT.VHTeddy_ice15, data)

In [ ]:
fice2eice_total_diff, eice2e_total_diff, e2zero_total_diff = calc_vertical(VHT.VHTDIFFbar_ice15 + VHT.VHTDIFFprime_ice15,
                                                                           data, diff=True)
fice2eice_bar_diff, eice2e_bar_diff, e2zero_bar_diff = calc_vertical(VHT.VHTDIFFbar_ice15, 
                                                                     data, diff=True)
fice2eice_prime_diff, eice2e_prime_diff, e2zero_prime_diff = calc_vertical(VHT.VHTDIFFprime_ice15, 
                                                                           data, diff=True)
fice2eice_eddy_diff, eice2e_eddy_diff, e2zero_eddy_diff = calc_vertical(VHT.VHTDIFFeddy_ice15, 
                                                                        data, diff=True)

In [ ]:
output["vertical across base of ML"] = {}
output["vertical across base of ML"]["80% - 15% sea ice"] = print_results(fice2eice_total, fice2eice_bar, 
                                                                          fice2eice_prime, fice2eice_eddy, 
                                                                          "adv. heat into the ML between 80% and 15% ice")

output["vertical across base of ML"]["y=80 - 80% sea ice"] = print_results(eice2e_total, eice2e_bar, 
                                                                           eice2e_prime, eice2e_eddy, 
                                                                           "adv. heat into the ML between y=80 and 80% ice")

output["vertical across base of ML"]["y=0 - y=80"] = print_results(e2zero_total, e2zero_bar, 
                                                                   e2zero_prime, e2zero_eddy, 
                                                                   "adv. heat into the ML between southern boundary and y=80")

In [ ]:
output["vertical diff. across base of ML"] = {}
output["vertical diff. across base of ML"]["80% - 15% sea ice"] = print_results(fice2eice_total_diff, fice2eice_bar_diff, 
                                                                          fice2eice_prime_diff, fice2eice_eddy_diff, 
                                                                          "diff. heat into the ML between 80% and 15% ice")

output["vertical diff. across base of ML"]["y=80 - 80% sea ice"] = print_results(eice2e_total_diff, eice2e_bar_diff, 
                                                                           eice2e_prime_diff, eice2e_eddy_diff, 
                                                                           "diff. heat into the ML between y=80 and 80% ice")

output["vertical diff. across base of ML"]["y=0 - y=80"] = print_results(e2zero_total_diff, e2zero_bar_diff, 
                                                                   e2zero_prime_diff, e2zero_eddy_diff, 
                                                                   "diff. heat into the ML between southern boundary and y=80")

In [ ]:
def calc_ice_ocean(data, ds):
    fifteenice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge15))).sum("YG")
    eightyice = (data.where(ds.YG < ds.YG.isel(YG=ice_edge80))).sum("YG")
    eighty = (data.where(ds.YG < ds.YG.isel(YG=80))).sum("YG")
    fifteenice2eightyice = fifteenice - eightyice
    eightyice2eighty = eightyice - eighty
    eighty2zero = eighty 
    return fifteenice2eightyice, eightyice2eighty, eighty2zero

In [ ]:
OIQfice2eice_total, OIQeice2e_total, OIQe2zero_total = calc_ice_ocean(OIQ.OIQtotal, data)
OIQfice2eice_bar, OIQeice2e_bar, OIQe2zero_bar = calc_ice_ocean(OIQ.OIQbar, data)
OIQfice2eice_prime, OIQeice2e_prime, OIQe2zero_prime = calc_ice_ocean(OIQ.OIQprime, data)
OIQfice2eice_eddy, OIQeice2e_eddy, OIQe2zero_eddy = calc_ice_ocean(OIQ.OIQtotaleddy, data)

In [ ]:
output["vertical from ocean to ice"] = {}
output["vertical from ocean to ice"]["80% - 15% sea ice"] = print_results(OIQfice2eice_total, OIQfice2eice_bar, 
                                                                          OIQfice2eice_prime, OIQfice2eice_eddy, 
                                                                          "heat into ice between 80 and 15 % ice")

output["vertical from ocean to ice"]["y=80 - 80% sea ice"] = print_results(OIQeice2e_total, OIQeice2e_bar, 
                                                                           OIQeice2e_prime, OIQeice2e_eddy, 
                                                                           "heat into ice between y=80 and 80% ice")

output["vertical from ocean to ice"]["y=0 - y=80"] = print_results(OIQe2zero_total, OIQe2zero_bar, 
                                                                   OIQe2zero_prime, OIQe2zero_eddy, 
                                                                   "heat into ice between y=80 and zero")

In [ ]:
if JJA:
    with open(path + 'post/table_of_fluxes.JJA.json', 'w', encoding='utf-8') as f: 
        json.dump(output, f, ensure_ascii=False, indent=4)
else:
    with open(path + 'post/table_of_fluxes.json', 'w', encoding='utf-8') as f: 
        json.dump(output, f, ensure_ascii=False, indent=4)